In this Jupyter notebook I cleaned my 9 text columns by tokenizing, removing stop words, stemming, etc, and then experimented with the best length of time between drug release and drug recall to predict on.  I returned csv files with training data and testing data to use for modeling in a separate notebook

In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('../datasets/NDC-drugs-with-recalls.csv')

In [58]:
df['clinical_pharmacology'][1]

"[u'12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Action The mechanism of action of carisoprodol in relieving discomfort associated with acute painful musculoskeletal conditions has not been clearly identified. In animal studies, muscle relaxation induced by carisoprodol is associated with altered interneuronal activity in the spinal cord and in the descending reticular formation of the brain. 12.2 Pharmacodynamics Carisoprodol is a centrally acting skeletal muscle relaxant that does not directly relax skeletal muscles. A metabolite of carisoprodol, meprobamate, has anxiolytic and sedative properties. The degree to which these properties of meprobamate contribute to the safety and efficacy of carisoprodol is unknown. 12.3 Pharmacokinetics The pharmacokinetics of carisoprodol and its metabolite meprobamate were studied in a crossover study of 24 healthy subjects (12 male and 12 female) who received single doses of 250 mg and 350 mg carisoprodol (see Table 2). The exposure of carisoprodol a

In [4]:
df.columns

Index([u'Unnamed: 0', u'Unnamed: 0_label', u'Unnamed: 0.1', u'abuse',
       u'accessories', u'active_ingredient', u'adverse_reactions', u'alarms',
       u'animal_pharmacology_and_or_toxicology', u'ask_doctor',
       ...
       u'good_ndcs', u'company_recall', u'product_recall', u'unix_label',
       u'unix_diff', u'diff_years', u'target', u'under_3', u'under_2',
       u'under_1'],
      dtype='object', length=107)

In [5]:
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop('Unnamed: 0_label', axis=1, inplace=True)
df.drop('Unnamed: 0.1', axis=1, inplace=True)


In [6]:
df.info()
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87251 entries, 0 to 87250
Columns: 104 entries, abuse to under_1
dtypes: float64(8), int64(77), object(19)
memory usage: 69.2+ MB


In [7]:
#for col in df.columns:

In [8]:
type(df['company_label'][0])

numpy.int64

In [9]:
int_cols=[]
nlp_cols=[]
float_cols=[]
for col in df.columns:
    if (type(df[col][0])==str):
        nlp_cols.append(col)
    elif (type(df[col][0])==np.int64):
        int_cols.append(col)
    elif (type(df[col][0])==np.float64):
        float_cols.append(col)
print int_cols,'\n',len(int_cols),'\n'
print nlp_cols,'\n',len(nlp_cols),'\n'
print float_cols,'\n',len(float_cols),'\n'
          
            

['abuse', 'accessories', 'alarms', 'animal_pharmacology_and_or_toxicology', 'ask_doctor', 'ask_doctor_or_pharmacist', 'assembly_or_installation_instructions', 'boxed_warning', 'calibration_instructions', 'carcinogenesis_and_mutagenesis_and_impairment_of_fertility', 'cleaning', 'clinical_studies', 'components', 'controlled_substance', 'dependence', 'diagram_of_device', 'disposal_and_waste_handling', 'do_not_use', 'dosage_forms_and_strengths', 'drug_abuse_and_dependence', 'drug_and_or_laboratory_test_interactions', 'drug_interactions', 'environmental_warning', 'food_safety_warning', 'general_precautions', 'geriatric_use', 'guaranteed_analysis_of_feed', 'health_care_provider_letter', 'health_claim', 'information_for_owners_or_caregivers', 'information_for_patients', 'instructions_for_use', 'intended_use_of_the_device', 'labor_and_delivery', 'laboratory_tests', 'mechanism_of_action', 'microbiology', 'nonclinical_toxicology', 'nonteratogenic_effects', 'nursing_mothers', 'other_safety_inform

In [10]:
int_cols.remove('target')

In [11]:
int_cols.remove('under_3')
int_cols.remove('under_2')
int_cols.remove('under_1')


In [12]:
len(int_cols)

73

In [13]:
int_cols.remove('company_label')
int_cols.remove('product_label')

In [14]:
nlp_cols.remove('NDCs')
nlp_cols.remove('unq_ndc')

In [15]:
print len(int_cols)
print len(nlp_cols)

71
9


In [16]:
nlp_cols


['adverse_reactions',
 'clinical_pharmacology',
 'contraindications',
 'description',
 'dosage_and_administration',
 'how_supplied',
 'indications_and_usage',
 'overdosage',
 'spl_product_data_elements']

In [17]:
for col in float_cols:
    print col,df[col][100]

Unnamed: 0_recall nan
unix nan
good_ndcs nan
company_recall nan
product_recall nan
unix_label 1269925200.0
unix_diff nan
diff_years nan


In [18]:
df.drop('Unnamed: 0_recall', axis=1, inplace=True)

In [19]:
print df['target'].sum()
print len(df['target'])
print df['under_3'].sum()
print df['under_2'].sum()
print df['under_1'].sum()


410
87251
410
283
140


In [20]:
print df['year'].value_counts()

2016    22349
2015    14441
2014    11066
2013     9198
2012     8136
2017     7162
2011     6717
2010     5759
2009     2230
2006       42
2008       34
2007       28
2004       16
2003       14
2005        8
2001        7
2000        7
2002        5
1998        5
1997        5
1986        3
1987        3
1996        3
1994        3
1993        2
1990        1
1989        1
1977        1
1991        1
1992        1
1999        1
1970        1
1984        1
Name: year, dtype: int64


In [21]:
predict_years=[2014,2015,2016]

In [22]:
df_test_train=df[(df['year'] !=2014) & (df['year'] !=2015) & (df['year'] !=2016)]

In [23]:
df_test_train['year'].unique()

array([2013, 2010, 2012, 2009, 2011, 2007, 2006, 2017, 2008, 1993, 1986,
       2004, 2003, 1990, 2001, 2002, 1997, 1989, 2005, 2000, 1998, 1994,
       1987, 1991, 1992, 1984, 1996, 1977, 1970, 1999])

In [24]:
df_predict=df[(df['year'] ==2014) | (df['year'] ==2015) | (df['year'] ==2016)]

In [25]:
df_predict['year'].unique()

array([2014, 2015, 2016])

In [26]:
print len(df_test_train)
print df_test_train['target'].sum()

39395
84


In [27]:
print len(df_predict)
print df_predict['target'].sum()

47856
326


In [28]:
df_1_yr_holdout=df[(df['year']!=2016) & (df['year']!=2017)]
df_2_yr_holdout=df[(df['year'] !=2016) & (df['year'] !=2015) & (df['year']!=2017)]
df_3_yr_holdout=df[(df['year'] !=2014) & (df['year'] !=2015) & (df['year'] !=2016) & (df['year']!=2017)]

In [29]:
print len(df_1_yr_holdout)
print df_1_yr_holdout['under_1'].sum() 
print len(df_2_yr_holdout)
print df_2_yr_holdout['under_2'].sum()
print len(df_3_yr_holdout)
print df_3_yr_holdout['under_3'].sum()


57740
107
43299
133
32233
58


In [30]:
print 107/57740.0
print 144/43299.0
print 84/32233.0

0.00185313474195
0.00332571191021
0.00260602488133


In [31]:
df_predict_1_year=df[ (df['year'] ==2016)]
df_predict_2_year=df[ (df['year'] ==2015) | (df['year'] ==2016)]
df_predict_3_year=df[(df['year'] ==2014) | (df['year'] ==2015) | (df['year'] ==2016)]

In [32]:
print len(df_predict_1_year)
print df_predict_1_year['under_1'].sum()
print len(df_predict_2_year)
print df_predict_2_year['under_2'].sum()
print len(df_predict_3_year)
print df_predict_3_year['under_3'].sum()

22349
29
36790
139
47856
326


In [33]:
print 29/22349.0
print 139/36790.0
print 326/47856.0

0.00129759720793
0.00377820059799
0.00681210297559


Here I select my training data to be all years up from 2010-2014, and to predict on the years 2015, 2016, and 2017, predicting if a drug will be recalled within 2 years of its release.  2015 and 16 don't have 2 years of data yet, so I held them out, and before 2009 there were very few drugs in the database of newly released drugs, and my recalls data only starts in 2012, so 2 years before in 2010 is the logical year to start the training

In [34]:
df_test_train=df[(df['year'] ==2009) |(df['year'] ==2010) |(df['year'] ==2011) |(df['year'] ==2012) |(df['year'] ==2013) |(df['year'] ==2014)]
df_predict=df[(df['year'] ==2015) |(df['year'] ==2016)|(df['year'] ==2017)]

In [35]:
df_test_train_no_2009=df[(df['year'] ==2010) |(df['year'] ==2011) |(df['year'] ==2012) |(df['year'] ==2013) |(df['year'] ==2014)]

In [36]:
print len(df_test_train)
print len(df_predict)
print len(df_test_train_no_2009)

43106
43952
40876


In [37]:
features=int_cols+nlp_cols

In [38]:
from sklearn.preprocessing import StandardScaler

In [39]:
df_std=StandardScaler().fit_transform(df[int_cols])

In [40]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [41]:
tokenizer = RegexpTokenizer(r'\w+')

In [42]:
test1=tokenizer.tokenize(df['adverse_reactions'][0].lower())

In [43]:
print len(test1)

908


In [44]:
stop=stopwords.words('english')

In [45]:
print len([i for i in test1 if i not in stop])

690


In [46]:
test3= [i for i in test1 if i in stop]

In [47]:
stemmed_text=[]
for item in test1:
    stemmer=PorterStemmer()
    new_item=stemmer.stem(item)
    stemmed_text.append(new_item)
print stemmed_text
    

['u', '6', u'advers', u'reaction', u'adult', 'most', 'common', u'advers', u'reaction', 'in', u'adult', u'incid', 'u2265', '2', 'are', u'headach', u'abdomin', 'pain', 'nausea', 'diarrhea', u'vomit', 'and', u'flatul', '6', u'pediatr', u'patient', '2', 'to', '16', u'year', 'of', 'age', u'safeti', u'profil', 'similar', 'to', 'that', 'in', u'adult', 'except', 'that', u'respiratori', 'system', u'event', 'and', 'fever', 'were', 'the', 'most', u'frequent', u'report', u'reaction', 'in', u'pediatr', u'studi', '8', '4', 'to', 'report', u'suspect', u'advers', u'reaction', 'contact', 'mylan', u'pharmaceut', 'inc', 'toll', 'free', 'at', '1', '877', '446', '3679', '1', '877', '4', 'info', 'rx', 'or', 'fda', 'at', '1', '800', 'fda', '1088', 'or', 'www', 'fda', 'gov', 'medwatch', '6', '1', u'clinic', u'trial', u'experi', 'with', u'omeprazol', u'monotherapi', u'becaus', u'clinic', u'trial', 'are', u'conduct', 'under', u'wide', u'vari', u'condit', u'advers', 'reaction', u'rate', u'observ', 'in', 'the', u

In [48]:
test4=stemmer.stem('hello everybody it\'s nicely to meeting y\'all')

In [49]:
print test4

hello everybody it's nicely to meeting y'al


In [50]:
def tokenize_text(text):
    #Remove Caps
    text_str=str(text)
    text_lower=text_str.lower()
    #Remove punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    no_punct=tokenizer.tokenize(text_lower)
    #Remove stopwords
    stop=stopwords.words('english')
    no_stopwords=[i for i in no_punct if i not in stop]
    #Stem words
    stemmer=PorterStemmer()
    stemmed_text=[]
    for item in no_stopwords:
        try:
            if (item == 'aed') | (item == 'aeds'):
                stemmed_text.append(item)
            else:
                new_item=stemmer.stem(item)
                stemmed_text.append(new_item)
        except:
            print item, type(item)

#         print len(no_stopwords)
    return ' '.join(stemmed_text)

In [51]:
tokenize_text(df['adverse_reactions'][0])

u'u 6 advers reaction adult common advers reaction adult incid u2265 2 headach abdomin pain nausea diarrhea vomit flatul 6 pediatr patient 2 16 year age safeti profil similar adult except respiratori system event fever frequent report reaction pediatr studi 8 4 report suspect advers reaction contact mylan pharmaceut inc toll free 1 877 446 3679 1 877 4 info rx fda 1 800 fda 1088 www fda gov medwatch 6 1 clinic trial experi omeprazol monotherapi clinic trial conduct wide vari condit advers reaction rate observ clinic trial drug cannot directli compar rate clinic trial anoth drug may reflect rate observ practic safeti data describ reflect exposur omeprazol delay releas capsul 3 096 patient worldwid clinic trial 465 patient u studi 2 631 patient intern studi indic clinic studi u trial includ duoden ulcer resist ulcer zolling ellison syndrom intern clinic trial doubl blind open label design common advers reaction report e incid rate u2265 2 omeprazol treat patient enrol studi includ headac

In [52]:
' '.join(tokenize_text(df['adverse_reactions'][0]))

u'u   6   a d v e r s   r e a c t i o n   a d u l t   c o m m o n   a d v e r s   r e a c t i o n   a d u l t   i n c i d   u 2 2 6 5   2   h e a d a c h   a b d o m i n   p a i n   n a u s e a   d i a r r h e a   v o m i t   f l a t u l   6   p e d i a t r   p a t i e n t   2   1 6   y e a r   a g e   s a f e t i   p r o f i l   s i m i l a r   a d u l t   e x c e p t   r e s p i r a t o r i   s y s t e m   e v e n t   f e v e r   f r e q u e n t   r e p o r t   r e a c t i o n   p e d i a t r   s t u d i   8   4   r e p o r t   s u s p e c t   a d v e r s   r e a c t i o n   c o n t a c t   m y l a n   p h a r m a c e u t   i n c   t o l l   f r e e   1   8 7 7   4 4 6   3 6 7 9   1   8 7 7   4   i n f o   r x   f d a   1   8 0 0   f d a   1 0 8 8   w w w   f d a   g o v   m e d w a t c h   6   1   c l i n i c   t r i a l   e x p e r i   o m e p r a z o l   m o n o t h e r a p i   c l i n i c   t r i a l   c o n d u c t   w i d e   v a r i   c o n d i t   a d v e r s   r e a c t i o 

In [53]:
# test10=tokenize_text(df['adverse_reactions'][len(df)-5])

In [54]:
#adverse_reactions_tok=df['adverse_reactions'].apply(tokenize_text)

In [55]:
#adverse_reactions_tok[0]

In [56]:
new_df=pd.DataFrame()

    

In [57]:
for col in nlp_cols:
    new_df[col]=df[col].apply(tokenize_text)

embryopathy <type 'str'>
known <type 'str'>


KeyboardInterrupt: 

In [ ]:
#print adverse_reactions_tok[0]

In [ ]:
from sklearn.externals import joblib
joblib.dump(new_df, 'tokenized_nlp_cols.pkl') 

In [ ]:
# from sklearn.model_selection import train_test_split

In [ ]:
# X_train,X_test,y_train,y_test=train_test_split(df_test_train[features],df_test_train['under_2'],test_size=0.3, random_state=None)

In [ ]:
new_df.columns

In [ ]:
len(new_df)

In [ ]:
df.columns

In [ ]:
int_cols

In [ ]:
float_cols

In [ ]:
for col in int_cols:
    new_df[col]=df[col]

In [ ]:
new_df['unq_ndc']=df['unq_ndc']

In [ ]:
new_df['target']=df['under_2']

In [ ]:
new_df.iloc[0,:]

In [ ]:
new_df['description'][0]

In [ ]:
df_test_train=new_df[(new_df['year'] ==2009) |(new_df['year'] ==2010) |(new_df['year'] ==2011) |(new_df['year'] ==2012) |(new_df['year'] ==2013) |(new_df['year'] ==2014)]
df_predict=new_df[(new_df['year'] ==2015) |(new_df['year'] ==2016)|(new_df['year'] ==2017)]
de_test_train_no_2009==new_df[(new_df['year'] ==2010) |(new_df['year'] ==2011) |(new_df['year'] ==2012) |(new_df['year'] ==2013) |(new_df['year'] ==2014)]

In [ ]:
df_test_train.to_csv('training_data_cleansed.csv')

In [ ]:
df_predict.to_csv('predicting_data_cleansed.csv')

In [ ]:
df_test_train_no_2009.to_csv('training_data_cleansed_no_2009.csv')

In [75]:
len(int_cols)

71